In [1]:
import numpy as np
from data_loader import load_data, DATA_PATH_TEST, DATA_PATH_TRAIN, DATA_PATH_SAMPLE_SUBMISSION_TEST
import logistic_regression
import gradient_descent
import matplotlib.pyplot as plt
from label_predictor import predict_labels

In [2]:
y, tx, ids_train = load_data(DATA_PATH_TRAIN)

In [3]:
def build_k_indices(y, number_of_subset, seed):
    rows_num = len(y)
    inter = int(rows_num / number_of_subset)
    #set the random seed
    np.random.seed(seed)
    indices = np.random.permutation(rows_num)

    subset_indices = []
    for i in range(number_of_subset):
        subset_indices.append(indices[i * inter: (i + 1) * inter])
    return np.array(subset_indices)

In [4]:
def cross_validation(y, tx, k_indices, k, gamma, alpha):
    #get the indices of the subsets
    test_set = k_indices[k]
    train_set = k_indices[~(np.arange(k_indices.shape[0]) == k)].reshape(-1)
    
    #get the subsets
    tx_train = tx[train_set]
    y_train = y[train_set]
    tx_test = tx[test_set]
    y_test = y[test_set]

    
    w, loss_train = logistic_regression.regularized_logistic_regression_gradient_descent(y_train, tx_train, gamma, 100,alpha)
    loss_test = logistic_regression.calculate_loss(y_test, tx_test, w)
    y_pred = predict_labels(w, tx_test)
    counter = 0
    #change the -1 to 0 to match y_test
    y_pred = [0 if x==-1 else x for x in y_pred]
    for i in range(y_test.shape[0]):
        if y_pred[i] == y_test[i]:
            counter += 1
    percent = 100*counter/y_test.shape[0]
#     print(percent)
    return loss_train, loss_test, w, percent

In [5]:
def build_poly(tx, degree):
    for idx, x in enumerate(tx.T):
        if idx == 0:
            arr_out = build_poly_one_column(x, degree)
        else:
            arr_out = np.c_[arr_out, build_poly_one_column(x, degree)]
    return arr_out

In [6]:
def build_poly_one_column(x, degree):
    arr = np.zeros((x.shape[0], degree+1))
    for degre in range(degree+1):
        arr[:,degre] = np.power(x, degre)
    return arr

In [7]:
def cross_validation_test():    
#     tx_train = np.delete(tx, [5, 12, 15, 18, 19, 20, 21, 23, 25, 27, 28, 29, 30], axis=1)
    for y_t, tx_t, id_test in zip(y, tx, ids_train):
        # split data in k fold
        number_of_subset = 4
        subset_indices = build_k_indices(y_t, number_of_subset, 12) #last number is the seed  
        percents = []
        tx_train = build_poly(tx_t, 2)
        for i in range(number_of_subset):
            _,_,_, percent = cross_validation(y_t, tx_train, subset_indices, i, 0.5,0.5)
            #print(percent)
            percents.append(percent)
        print(np.mean(percents))

cross_validation_test()


ValueError: shapes (19590,1) and (19590,1) not aligned: 1 (dim 1) != 19590 (dim 0)

In [9]:
def cross_validation_search_param(tx, y):
    # split data in k fold
    number_of_subset = 4
    subset_indices = build_k_indices(y, number_of_subset, 12) #last number is the seed
    
    #Best parameter
    bestW = []
    bestRatio = 0
    bestG = 0
    bestA = 0
    bestD = 0
    
    count = 0
    #Test of different degrees
    for d in np.arange(2, 3, 1):  
        tx_train = build_poly(tx, d)
        #Test of different alpha
        for a in np.arange(0.1, 1, 0.1):
            #Test of different gamma
            for g in np.arange(0.1, 1, 0.1):
                # define lists to store the ratio of true mapping
                ratio = 0
                ratios= []
                for k in range(number_of_subset):
                    _, _, w, ratio = cross_validation(y, tx_train, subset_indices, k, g, a)
                    ratios.append(ratio)
                if np.mean(ratios) > bestRatio:
                    print("oui")
                    bestW = w
                    bestG = g
                    bestA = a
                    bestD = d
                    bestRatio = np.mean(ratios)
                count += 1
#                 if(count%100 == 0):
#                     print(count)
#                 print("ratio:", np.mean(ratios))
    print("bestRatio:", bestRatio)
    return bestW, bestG, bestA, bestD
counter = 0
for y_t, tx_t, id_test in zip(y, tx, ids_train):
    bestW, bestG, bestA, bestD = cross_validation_search_param(tx_t, y_t)
    print("Set ", counter)
    print("bestW ", bestW)
    print("bestG ", bestG)
    print("bestA ", bestA)
    print("bestD ", bestD)
    counter += 1

D:\Users\benja\git\MLProjet\epfml\src\logistic_regression.py:11: RuntimeWarning: divide by zero encountered in log
  loss_2 = np.matmul((1 - y).T,(np.log(1 - sigma)))


oui


D:\Users\benja\git\MLProjet\epfml\src\logistic_regression.py:10: RuntimeWarning: divide by zero encountered in log
  loss_1 = -np.matmul(y.T,(np.log(sigma)))


oui
oui
bestRatio: 94.73583460949465
Set  0
bestW  [-0.13355594 -0.13355594 -0.13355594 -0.13355594  0.17675639  0.08146928
 -0.13355594  0.02794928 -0.46874209 -0.13355594 -0.00423898 -0.05927714
 -0.13355594 -0.03842175 -0.1961759  -0.13355594 -0.00423926 -0.05927741
 -0.13355594  0.41947964  0.18235025 -0.13355594 -0.30889708 -0.01150863
 -0.13355594 -0.051237   -0.00124135 -0.13355594  0.47128012 -0.1389376
 -0.13355594  0.03245365  0.00316217 -0.13355594 -0.01646251 -0.10537716
 -0.13355594  0.18534134 -0.20089854 -0.13355594  0.03745565 -0.14220587
 -0.13355594  0.01336344 -0.075681   -0.13355594  0.24087855  0.07977812
 -0.13355594  0.02979753 -0.08491963 -0.13355594 -0.02137331 -0.08085034]
bestG  0.30000000000000004
bestA  0.2
bestD  2
oui
bestRatio: 91.37566137566138
Set  1
bestW  [-0.0839546  -0.0839546  -0.0839546  -0.0839546   0.16003053 -0.01659541
 -0.0839546   0.03245236 -0.33187928 -0.0839546  -0.13981232  0.03854788
 -0.0839546  -0.05039938 -0.18021428 -0.0839546   0.

KeyboardInterrupt: 